In [3]:
from torch_geometric.datasets import Planetoid

import numpy as np
import random
from random import shuffle
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from qiskit import *

import torch.optim as optim
from torch.nn.parameter import Parameter
from sklearn.metrics import f1_score
import scipy.sparse as sp
import numpy as np
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.nn import GCNConv
import sys
from givens_roatation import GivensRotations
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchquantum as tq
#import torchquantum.layers
#from tq.layers import Op1QAllLayer,Op2QAllLayer
from measure import Measure
from qlayers import QLayer_block1,QLayer_block2,QLayer_block3,QLayer_block4
import os
import matplotlib.pyplot as plt  
import seaborn as sns  

In [4]:
from SeversonDataset_preprocess import mat_to_npy 

mat_to_npy() # load mat files and delete some samples with anomalies

skip err cell: batch 1, cell_id 0
skip err cell: batch 1, cell_id 1
skip err cell: batch 1, cell_id 2
skip err cell: batch 1, cell_id 3
skip err cell: batch 1, cell_id 4
skip err cell: batch 1, cell_id 5
(4, 500)
(4, 500)
1
(4, 500)
(4, 500)
2
(4, 500)
(4, 500)
3
(4, 500)
(4, 500)
4
(4, 500)
(4, 500)
5
(4, 500)
(4, 500)
6
(4, 500)
(4, 500)
7
(4, 500)
(4, 500)
8
(4, 500)
(4, 500)
9
(4, 500)
(4, 500)
10
(4, 500)
(4, 500)
11
(4, 500)
(4, 500)
12
(4, 500)
(4, 500)
13
(4, 500)
(4, 500)
14
(4, 500)
(4, 500)
15
(4, 500)
(4, 500)
16
(4, 500)
(4, 500)
17
(4, 500)
(4, 500)
18
(4, 500)
(4, 500)
19
(4, 500)
(4, 500)
20
(4, 500)
(4, 500)
21
(4, 500)
(4, 500)
22
(4, 500)
(4, 500)
23
(4, 500)
(4, 500)
24
(4, 500)
(4, 500)
25
(4, 500)
(4, 500)
26
(4, 500)
(4, 500)
27
(4, 500)
(4, 500)
28
(4, 500)
(4, 500)
29
(4, 500)
(4, 500)
30
(4, 500)
(4, 500)
31
(4, 500)
(4, 500)
32
(4, 500)
(4, 500)
33
(4, 500)
(4, 500)
34
(4, 500)
(4, 500)
35
(4, 500)
(4, 500)
36
(4, 500)
(4, 500)
37
(4, 500)
(4, 500)
38
(4, 500

In [5]:
from data_preprocessing import load_Severson
trn_feature, trn_target = load_Severson(training=True, norm=False, part='discharge')

In [6]:
print(trn_feature.shape)
print(trn_target.shape)

(4550, 4, 500)
(4550, 3)


In [7]:
# Reshape the training data from (4550, 4, 500) to (91, 100000)
# 4550 = 91 batteries * 50 cycles, reshaping cycles and features
num_batteries = 91
num_features = trn_feature.shape[1]
data_per_cycle = trn_feature.shape[2]
training_data_reshaped = trn_feature.reshape(num_batteries, -1)  # (91, 100000)

# Extract the last target ([:,2]) and reshape into (91, 1)
# Ensure only one target per battery by taking the first target for each set of 50 cycles
training_target_reshaped = trn_target[::50, 2].reshape(num_batteries,-1)  # (91, 1)

In [8]:
print(training_data_reshaped.shape)
print(training_target_reshaped.shape)

(91, 100000)
(91, 1)


In [9]:
#print(torch.flatten(training_target_reshaped))

In [10]:
np.save("train", training_data_reshaped)
np.save("target", training_target_reshaped)

In [11]:
import torch
from torch_geometric.data import Data, InMemoryDataset

#The class is turning the battery data into graph dataset for easier implentation of QGCN
class BatteryGraphDataset(InMemoryDataset):
    def __init__(self, data, labels, transform=None):
        """
        :param data: Tensor of shape [91, 50 * 500 * 4] (batteries, nodes, features)
        :param labels: Tensor of shape [91, 1] (batteries, labels)
        :param transform: (Optional) Data transformation function
        """
        self.data_array = data
        self.label_array = labels
        super().__init__(None, transform)

        # Prepare data list (all data in dataset[0])
        data_list = []
        for i in range(data.size(0)):  # For each battery
            x = data[i].reshape(-1, 4)  # [50*500, 4 features]
            y = labels[i].repeat(x.size(0),1)  # Repeat label for each node
            data_obj = Data(x=x, y=y)
            data_list.append(data_obj)

        # Combine all graphs into dataset[0]
        combined_x = torch.cat([d.x for d in data_list], dim=0)  # Combine all x
        combined_y = torch.cat([d.y for d in data_list], dim=0)  # Combine all y

        # Truncate data to match the target size
        target_size = 65536  # Desired number of rows
        if combined_x.size(0) > target_size:
            combined_x = combined_x[:target_size]
            combined_y = combined_y[:target_size]
        
        # Create train and test masks
        train_ratio = 0.8
        num_nodes = combined_x.size(0)
        num_train = int(train_ratio * num_nodes)
        train_mask = torch.zeros(num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(num_nodes, dtype=torch.bool)
        train_mask[:num_train] = True
        test_mask[num_train:] = True

        # Create edge_index in COO format (within-cycle connections)
        edge_index = []
        time_points = 500
        cycles_per_battery = 50
        nodes_per_battery = cycles_per_battery * time_points

        for battery_idx in range(data.size(0)):
            for cycle_idx in range(cycles_per_battery):
                start_idx = battery_idx * nodes_per_battery + cycle_idx * time_points
                for t in range(time_points - 1):
                    edge_index.append([start_idx + t, start_idx + t + 1])  # Forward edge
                    edge_index.append([start_idx + t + 1, start_idx + t])  # Backward edge

        # Convert edge list to tensor
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

        max_edges = 65536
        if edge_index.size(1) > max_edges:
            edge_index = edge_index[:, :max_edges]

        # Create a single graph object
        self.data = Data(
            x=combined_x.float(),
            #y=combined_y,
            y=torch.flatten(combined_y.int().long()),
            train_mask=train_mask,
            test_mask=test_mask,
            edge_index=edge_index
        )

    def len(self):
        return 1  # Single dataset entry

    def get(self, idx):
        assert idx == 0, "Only a single graph stored in this dataset."
        return self.data


# Example usage
# Simulate reshaped data
num_batteries = 91
num_cycles = 50
num_points = 500
num_features = 4

# Flattened data with 2275000 rows (91 batteries × 50 cycles × 500 points)
#training_data_reshaped = torch.rand(num_batteries, num_cycles * num_points * num_features)
#training_target_reshaped = torch.randint(0, 2, (1,num_batteries)) # Binary labels for each battery

training_data_reshaped = np.load("train.npy")
training_data_reshaped = torch.from_numpy(training_data_reshaped)
training_target_reshaped = np.load("target.npy") # Binary labels for each battery
training_target_reshaped = torch.from_numpy(training_target_reshaped)
training_target_reshaped = torch.flatten(training_target_reshaped)
print("Node Features Shape:", training_data_reshaped.shape)  # Should be [2097152, 4]
print("Labels Shape:", training_target_reshaped.shape)  
#training_target_reshaped = labels
# Create the dataset
dataset = BatteryGraphDataset(training_data_reshaped, training_target_reshaped)

# Check the combined graph
data_sample = dataset[0]
print("Node Features Shape:", data_sample.x)  # Should be [2097152, 4]
print("Labels Shape:", data_sample.y)         # Should be [2097152, 1]


Node Features Shape: torch.Size([91, 100000])
Labels Shape: torch.Size([91])
Node Features Shape: tensor([[-1.7530, -1.7512, -1.7495, -1.7495],
        [-1.7495, -1.7495, -1.7491, -1.7471],
        [-1.7456, -1.7456, -1.7450, -1.7431],
        ...,
        [-0.7413, -0.7294, -0.7209, -0.7209],
        [-0.7100, -0.7045, -0.7027, -0.6909],
        [-0.6882, -0.6835, -0.6719, -0.6719]])
Labels Shape: tensor([0, 0, 0,  ..., 0, 0, 0])


/Users/chenying/miniconda3/envs/torchquantum/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [12]:
dataset = BatteryGraphDataset(training_data_reshaped, training_target_reshaped)

In [13]:
print(dataset[0])

Data(x=[65536, 4], edge_index=[2, 65536], y=[65536], train_mask=[65536], test_mask=[65536])


In [14]:
data = dataset[0]
n_node =  65536
n_train = int(n_node/2)
n_feature = 4
n_vqc_wires = int(np.log2(n_feature))
#label0 = 2
#label1 = 3

In [15]:
features = data.x
labels = data.y
features = torch.nn.functional.normalize(features, p=2, dim=0)
print('features original shape:',features.shape)

features original shape: torch.Size([65536, 4])


In [16]:
print(labels)

tensor([0, 0, 0,  ..., 0, 0, 0])


In [17]:
shuffle_index = np.array(range(labels.shape[0]))
print(shuffle_index)

[    0     1     2 ... 65533 65534 65535]


In [18]:
data = data.subgraph(torch.tensor(shuffle_index[0:n_node]))

In [19]:
train_index = torch.zeros_like(data.train_mask)
train_index[0:n_train] = 1
train_index = train_index.to(torch.bool)
test_index = torch.zeros_like(data.test_mask)
test_index[n_train:n_node] =1
test_index = test_index.to(torch.bool)
print(train_index.sum(),test_index.sum())
print(train_index.shape)
print(test_index.shape)

tensor(32768) tensor(32768)
torch.Size([65536])
torch.Size([65536])


In [20]:
nclass = labels.max().item()+1
edges = data.edge_index.T


print('edge.shape:',edges.shape)
pq_series = []

for i in range(edges.shape[0]):
    # if edges[i][0].item() in range(n_train,n_node) and edges[i][1].item() in range(n_train,n_node):
    #     continue
    if edges[i][0].item() < edges[i][1].item():
        pq_series.append([edges[i][0].item(), edges[i][1].item()])


edge_index = torch.tensor(pq_series).T
# data.edge_index = edge_index
for i in range(int(max(labels))+1):
    print( 'label = ',i,'sum =',(labels==i).sum())

print('edge.shape:',edge_index.shape)

# sys.exit(0)
print(nclass)
print(features.shape)
print(labels.shape)

edge.shape: torch.Size([65536, 2])
label =  0 sum = tensor(40536)
label =  1 sum = tensor(25000)
edge.shape: torch.Size([2, 32768])
2
torch.Size([65536, 4])
torch.Size([65536])


In [21]:
def normalize_adj(mx):

    mx = mx + sp.eye(mx.shape[0])
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1/2).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    mx = mx.dot(r_mat_inv)
    return mx

In [22]:
class GCN(torch.nn.Module):
    def __init__(self, nfeat,  nclass,edge_index):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(nfeat, nclass, bias= False)
        self.edge_index = edge_index

    def forward(self, x):
        # x, edge_index = data.x, data.edge_index # 取出x, edge_index
        x = self.conv1(x, self.edge_index)
        return x

In [23]:
class QGCN(tq.QuantumModule):
    def __init__(self,g,n_nodes,n_dimension,pqs) :
        super().__init__()
        self.n_qubit =math.ceil(math.log(n_nodes,2))
        self.qgcn1 = GivensRotations(self.n_qubit,pqs)
        # self.qgcn1.genTrans()
        # self.qgcn2 = GivensRotations(self.n_qubit,pqs)
        self.linear1 = torch.nn.Linear(int(np.log2(n_feature)),2)
        # self.linear2 = torch.nn.Linear(32,2)
        self.adj = to_scipy_sparse_matrix(g.edge_index,num_nodes=n_nodes)
        self.adj = normalize_adj(self.adj)
        self.adj =torch.tensor(self.adj.todense(),dtype= torch.float32)
        self.n_wires = n_vqc_wires
        self.q_layer1 = QLayer_block4(self.n_wires)
        self.q_layer2 = QLayer_block4(self.n_wires)
        self.q_layer3 = QLayer_block4(self.n_wires)
        self.q_layer4 = QLayer_block4(self.n_wires)
        self.q_layer5 = QLayer_block4(self.n_wires)
        self.q_layer6 = QLayer_block4(self.n_wires)
        self.q_layer7 = QLayer_block4(self.n_wires)
        self.q_layer8 = QLayer_block4(self.n_wires)
        self.q_layer9 = QLayer_block4(self.n_wires)
        self.q_layer10 = QLayer_block4(self.n_wires)
        self.q_layer11 = QLayer_block4(self.n_wires)
        self.q_layer12 = QLayer_block4(self.n_wires)
        self.q_layer13 = QLayer_block4(self.n_wires)

        self.encoder = tq.StateEncoder()
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        self.measure = Measure(obs=tq.PauliZ,wires= range(self.n_wires))
        # self.n_dimension = n_dimension
    def compare(self):
        adj =self.adj
        givens = self.qgcn1.get_trans()
        return adj, givens

    def forward(self,h):
        bsz = h.shape[0]
        h = self.qgcn1(h)
        #h = torch.mm(self.adj,h)
        #h = h*h
        h = self.encoder(self.q_device,h)
        self.q_layer1(self.q_device)
        self.q_layer2(self.q_device)
        self.q_layer3(self.q_device)
        self.q_layer4(self.q_device)
        self.q_layer5(self.q_device)
        self.q_layer6(self.q_device)
        self.q_layer7(self.q_device)
        #self.q_layer8(self.q_device)
        #self.q_layer9(self.q_device)
        #self.q_layer10(self.q_device)
        #self.q_layer11(self.q_device)
        h = self.measure(self.q_device)
        #print(h.size())
        #print(h)
        h = self.linear1(h)
        #h = h.reshape(bsz, 2, -1).sum(-1).squeeze()
        return h

    def quantum_inference(self,input):
        pass
    # def show_circuit(self):
    #     print(self.qgcn1.circuit)

In [24]:
loss_func = torch.nn.CrossEntropyLoss()


def train(model, data, train_index, lr=0.01, weight_decay=5e-4, epochs=20):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    labels = data.y
    train_mask = train_index

    best_loss_val = 100
    
    for i in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(data.x)
        # print(output.shape)
        loss = loss_func(output[train_mask], labels[train_mask]) # 用训练集中的节点来计算损失函数
        # print(loss)

        loss.backward()
        optimizer.step()
        scheduler.step()
        if i % 10 == 0:
            # print('parameters:',list(model.parameters())[1].item())
            print('Epoch {}, training loss: {}'.format(i, loss.item()))

@torch.no_grad()
def test(model, data,test_index):
    """Evaluate GAT performance on test set.

    """
    model.eval()
    test_mask = test_index
    labels = data.y 
    output = model(data.x) # 得到模型输出

    loss_test = loss_func(output[test_mask], labels[test_mask])
    preds = output[test_mask].argmax(1) # 得到预测值
   
    # print(preds)
    # preds = preds.argmax(1)
    # preds = preds > 0.5
    # print(labels[test_mask])
    # labels = labels > 0.5
    # print(preds) 
    # print(labels[test_mask])
    # sys.exit(0)
    acc_test = preds.eq(labels[test_mask]).cpu().numpy().mean() # 得到准确率
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test))
    return preds, output, acc_test.item()

In [ ]:
#model = GCN(nfeat=features.shape[1], nclass=nclass,edge_index= data.edge_index)
model = QGCN(data,n_nodes=n_node,n_dimension=4,pqs=edge_index.T)


device = 'cpu'
model = model.to(device)
data = data.to(device)
train(model, data,train_index=train_index, epochs=100)
#edge_index and shuffle index

num: 32768


In [ ]:
preds, output, acc = test(model, data,test_index=test_index)

Test set results: loss= 0.1640 accuracy= 0.9980
